In [ ]:
## This file implements neural networks with/without dropout and regularizer for p002ykpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [89]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [90]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p002ykpresabsSTCC_qual.csv')
df.shape

(253, 1738)

In [91]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [92]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [93]:
df.head()

,id,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,...,AACCTTAAAATTAACTTCTTTCAATTTCATAATAAAGTCTCTATAAAATAACTTAGTTTAAAAACGATTCGTATCTTTCAGATTCAAATACCATCATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATGCAGTAAAATTAATCGAGAGGTGCCATTTTGAAGACAGGTCGAATAGTGAAATCAATTAGTGGGGTATATCAAGTAGACGTTAATGGCGAACGTTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,ST,CC,pheno
0,107,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,1,0,5,5,0
1,109,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,1,0,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,5,5,1
3,120335,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,1,0,5,5,0
4,120337,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,1,0,5,5,0


In [94]:
df['pheno'].value_counts()

0    220
1     30
2      3
Name: pheno, dtype: int64

In [95]:
df_clean = df.drop(columns=['id'])

In [96]:
df_clean.shape

(253, 1737)

In [97]:
df_clean.head()

,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,...,AACCTTAAAATTAACTTCTTTCAATTTCATAATAAAGTCTCTATAAAATAACTTAGTTTAAAAACGATTCGTATCTTTCAGATTCAAATACCATCATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATGCAGTAAAATTAATCGAGAGGTGCCATTTTGAAGACAGGTCGAATAGTGAAATCAATTAGTGGGGTATATCAAGTAGACGTTAATGGCGAACGTTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,ST,CC,pheno
0,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,1,0,5,5,0
1,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,1,0,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,5,5,1
3,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,1,0,5,5,0
4,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,1,0,5,5,0


In [98]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 1737) (253,)


In [99]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 220), (1, 220), (2, 220)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [100]:
############# Fully-Connected Neural Network ################

In [123]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [124]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [125]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [126]:
dat

,0,test
0,CFBRSa26,0
1,NRS109,2
2,NRS112,0
3,NRS216,1
4,NRS021,0
...,...,...
193,CFBREBSa133,0
194,NRS209,2
195,NRS109,2
196,NRS209,2


In [127]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 391us/step - loss: 1.0039 - accuracy: 0.5996 - val_loss: 0.8888 - val_accuracy: 0.7677
Epoch 2/100
462/462 [==============================] - 0s 174us/step - loss: 0.5517 - accuracy: 0.7706 - val_loss: 0.5731 - val_accuracy: 0.7727
Epoch 3/100
462/462 [==============================] - 0s 248us/step - loss: 0.5479 - accuracy: 0.7662 - val_loss: 0.5057 - val_accuracy: 0.7727
Epoch 4/100
462/462 [==============================] - 0s 550us/step - loss: 0.4180 - accuracy: 0.8052 - val_loss: 0.4701 - val_accuracy: 0.7475
Epoch 5/100
462/462 [==============================] - 0s 371us/step - loss: 0.4223 - accuracy: 0.7987 - val_loss: 0.4691 - val_accuracy: 0.7323
Epoch 6/100
462/462 [==============================] - 0s 317us/step - loss: 0.4183 - accuracy: 0.8095 - val_loss: 0.5513 - val_accuracy: 0.7626
Epoch 7/100
462/462 [==============================] - 0s 377us/step - loss: 0.4133 

Epoch 57/100
462/462 [==============================] - 0s 240us/step - loss: 0.1734 - accuracy: 0.9416 - val_loss: 0.1571 - val_accuracy: 0.9394
Epoch 58/100
462/462 [==============================] - 0s 174us/step - loss: 0.1711 - accuracy: 0.9264 - val_loss: 0.1538 - val_accuracy: 0.9192
Epoch 59/100
462/462 [==============================] - 0s 183us/step - loss: 0.1710 - accuracy: 0.9242 - val_loss: 0.1758 - val_accuracy: 0.9192
Epoch 60/100
462/462 [==============================] - 0s 173us/step - loss: 0.2188 - accuracy: 0.9069 - val_loss: 0.1644 - val_accuracy: 0.9192
Epoch 61/100
462/462 [==============================] - 0s 149us/step - loss: 0.1693 - accuracy: 0.9416 - val_loss: 0.1523 - val_accuracy: 0.9293
Epoch 62/100
462/462 [==============================] - 0s 149us/step - loss: 0.1679 - accuracy: 0.9372 - val_loss: 0.1473 - val_accuracy: 0.9192
Epoch 63/100
462/462 [==============================] - 0s 195us/step - loss: 0.1582 - accuracy: 0.9437 - val_loss: 0.1780 -

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

198/198 [==============================] - 0s 156us/step
over-sampling test accuracy: 93.43%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 2, 0, 0, 0, 2, 2, 0, 0, 1,
       0, 1, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 0, 0, 2, 1, 1, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 0, 0,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1,
       0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 2, 0, 2, 2, 2, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa26,0,0
1,NRS109,2,2
2,NRS112,0,0
3,NRS216,1,1
4,NRS021,0,0
...,...,...,...
193,CFBREBSa133,0,0
194,NRS209,2,2
195,NRS109,2,2
196,NRS209,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,6.236406e-01,0.376359,3.521993e-07
1,7.026161e-03,0.015500,9.774740e-01
2,9.819342e-01,0.018031,3.455183e-05
3,8.183586e-02,0.916974,1.189723e-03
4,9.957193e-01,0.004281,9.797663e-12
...,...,...,...
193,9.202144e-01,0.062758,1.702757e-02
194,8.530889e-11,0.000054,9.999461e-01
195,7.026157e-03,0.015500,9.774740e-01
196,8.530889e-11,0.000054,9.999461e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p002ST.csv", index = False,
         header=None)

In [24]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 221us/step - loss: 0.1300 - accuracy: 0.9416 - val_loss: 0.1505 - val_accuracy: 0.9444
Epoch 2/100
462/462 [==============================] - 0s 185us/step - loss: 0.1266 - accuracy: 0.9545 - val_loss: 0.1290 - val_accuracy: 0.9394
Epoch 3/100
462/462 [==============================] - 0s 173us/step - loss: 0.1517 - accuracy: 0.9351 - val_loss: 0.1825 - val_accuracy: 0.9141
Epoch 4/100
462/462 [==============================] - 0s 176us/step - loss: 0.1369 - accuracy: 0.9459 - val_loss: 0.1369 - val_accuracy: 0.9444
Epoch 5/100
462/462 [==============================] - 0s 177us/step - loss: 0.1405 - accuracy: 0.9372 - val_loss: 0.1762 - val_accuracy: 0.9394
Epoch 6/100
462/462 [==============================] - 0s 175us/step - loss: 0.1478 - accuracy: 0.9394 - val_loss: 0.1323 - val_accuracy: 0.9343
Epoch 7/100
462/462 [==============================] - 0s 185us/step - loss: 0.1293 

Epoch 57/100
462/462 [==============================] - 0s 207us/step - loss: 0.1287 - accuracy: 0.9481 - val_loss: 0.1933 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 184us/step - loss: 0.1301 - accuracy: 0.9545 - val_loss: 0.1624 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 176us/step - loss: 0.1020 - accuracy: 0.9610 - val_loss: 0.2237 - val_accuracy: 0.9293
Epoch 60/100
462/462 [==============================] - 0s 179us/step - loss: 0.1182 - accuracy: 0.9610 - val_loss: 0.1541 - val_accuracy: 0.9394
Epoch 61/100
462/462 [==============================] - 0s 191us/step - loss: 0.1081 - accuracy: 0.9437 - val_loss: 0.1606 - val_accuracy: 0.9394
Epoch 62/100
462/462 [==============================] - 0s 174us/step - loss: 0.1047 - accuracy: 0.9610 - val_loss: 0.1473 - val_accuracy: 0.9394
Epoch 63/100
462/462 [==============================] - 0s 191us/step - loss: 0.1050 - accuracy: 0.9610 - val_loss: 0.1774 -

In [25]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 94.75%


In [128]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [129]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [130]:
y_prob = df_proba[df_proba['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[6.23640600e-01, 3.76359050e-01, 3.52199320e-07],
       [7.02616060e-03, 1.54997570e-02, 9.77474030e-01],
       [9.81934200e-01, 1.80312170e-02, 3.45518350e-05],
       [8.18358600e-02, 9.16974370e-01, 1.18972260e-03],
       [9.95719250e-01, 4.28071400e-03, 9.79766300e-12],
       [6.23640600e-01, 3.76359050e-01, 3.52199320e-07],
       [6.01573000e-05, 2.72155160e-03, 9.97218250e-01],
       [6.01573000e-05, 2.72155160e-03, 9.97218250e-01],
       [7.02616060e-03, 1.54997570e-02, 9.77474030e-01],
       [9.95745360e-01, 4.25468200e-03, 3.33418980e-09],
       [3.68795430e-03, 9.95647600e-01, 6.64382800e-04],
       [9.95719250e-01, 4.28071400e-03, 9.79766300e-12],
       [3.01927700e-02, 9.69804940e-01, 2.32754590e-06],
       [2.35658030e-02, 8.74257900e-01, 1.02176210e-01],
       [9.99265250e-01, 7.34733530e-04, 1.14765090e-12],
       [7.49473130e-03, 9.92497740e-01, 7.49180440e-06],
       [6.01573000e-05, 2.72155160e-03, 9.97218250e-01],
       [1.06678610e-03, 9.98655

In [131]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [132]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9933807774716866

In [133]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [134]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9933807774716866

In [135]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [136]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [137]:
dat2

,0,test
0,NRS109,2
1,NRS109,2
2,NRS222,0
3,NRS109,2
4,GA50245,0
...,...,...
193,NRS148,2
194,NRS266,1
195,NRS109,2
196,NRS149,0


In [138]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 467us/step - loss: 1.1895 - accuracy: 0.5714 - val_loss: 0.7791 - val_accuracy: 0.6212
Epoch 2/100
462/462 [==============================] - 0s 257us/step - loss: 0.5415 - accuracy: 0.7511 - val_loss: 0.5956 - val_accuracy: 0.7778
Epoch 3/100
462/462 [==============================] - 0s 248us/step - loss: 0.5255 - accuracy: 0.8030 - val_loss: 0.5287 - val_accuracy: 0.7879
Epoch 4/100
462/462 [==============================] - 0s 238us/step - loss: 0.4116 - accuracy: 0.8160 - val_loss: 0.4545 - val_accuracy: 0.8030
Epoch 5/100
462/462 [==============================] - 0s 270us/step - loss: 0.3567 - accuracy: 0.8550 - val_loss: 0.4191 - val_accuracy: 0.8081
Epoch 6/100
462/462 [==============================] - 0s 204us/step - loss: 0.3453 - accuracy: 0.8745 - val_loss: 0.4101 - val_accuracy: 0.8535
Epoch 7/100
462/462 [==============================] - 0s 231us/step - loss: 0.3158 

Epoch 57/100
462/462 [==============================] - 0s 221us/step - loss: 0.1478 - accuracy: 0.9610 - val_loss: 0.2455 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 156us/step - loss: 0.1412 - accuracy: 0.9545 - val_loss: 0.1878 - val_accuracy: 0.9242
Epoch 59/100
462/462 [==============================] - 0s 174us/step - loss: 0.1341 - accuracy: 0.9567 - val_loss: 0.2141 - val_accuracy: 0.9242
Epoch 60/100
462/462 [==============================] - 0s 211us/step - loss: 0.1560 - accuracy: 0.9459 - val_loss: 0.2235 - val_accuracy: 0.9242
Epoch 61/100
462/462 [==============================] - 0s 384us/step - loss: 0.1176 - accuracy: 0.9589 - val_loss: 0.2111 - val_accuracy: 0.9293
Epoch 62/100
462/462 [==============================] - 0s 413us/step - loss: 0.1489 - accuracy: 0.9394 - val_loss: 0.2589 - val_accuracy: 0.9343
Epoch 63/100
462/462 [==============================] - 0s 183us/step - loss: 0.1498 - accuracy: 0.9524 - val_loss: 0.2508 -

In [33]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

198/198 [==============================] - 0s 152us/step
over-sampling test accuracy: 92.93%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 1,
       2, 1, 1, 2, 1, 2, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1,
       2, 1, 0, 2, 1, 1, 1, 2, 0, 1, 0, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2,
       1, 0, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 1, 1, 1, 1, 0, 1, 2, 0, 1, 0,
       2, 0, 2, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1,
       0, 2, 1, 1, 0, 1, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0, 1, 1,
       1, 1, 2, 2, 1, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 0,
       0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 2, 0, 2])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS109,2,2
1,NRS109,2,2
2,NRS222,0,0
3,NRS109,2,2
4,GA50245,0,0
...,...,...,...
193,NRS148,2,2
194,NRS266,1,1
195,NRS109,2,2
196,NRS149,0,0


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.015403,0.017533,9.670641e-01
1,0.015403,0.017533,9.670641e-01
2,0.951918,0.048061,2.067555e-05
3,0.015403,0.017533,9.670641e-01
4,0.940524,0.037246,2.223014e-02
...,...,...,...
193,0.000114,0.002016,9.978700e-01
194,0.003898,0.995024,1.078068e-03
195,0.015403,0.017533,9.670641e-01
196,0.717245,0.282755,2.331203e-10


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p002ST.csv", index = False,
         header=None)

In [37]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 220us/step - loss: 0.1215 - accuracy: 0.9545 - val_loss: 0.2041 - val_accuracy: 0.9242
Epoch 2/100
462/462 [==============================] - 0s 208us/step - loss: 0.1427 - accuracy: 0.9372 - val_loss: 0.2095 - val_accuracy: 0.9293
Epoch 3/100
462/462 [==============================] - 0s 214us/step - loss: 0.1362 - accuracy: 0.9437 - val_loss: 0.2087 - val_accuracy: 0.9343
Epoch 4/100
462/462 [==============================] - 0s 197us/step - loss: 0.1147 - accuracy: 0.9632 - val_loss: 0.2028 - val_accuracy: 0.9343
Epoch 5/100
462/462 [==============================] - 0s 184us/step - loss: 0.1134 - accuracy: 0.9654 - val_loss: 0.1875 - val_accuracy: 0.9293
Epoch 6/100
462/462 [==============================] - 0s 234us/step - loss: 0.1357 - accuracy: 0.9610 - val_loss: 0.1991 - val_accuracy: 0.9343
Epoch 7/100
462/462 [==============================] - 0s 199us/step - loss: 0.1385 

Epoch 57/100
462/462 [==============================] - 0s 170us/step - loss: 0.0996 - accuracy: 0.9589 - val_loss: 0.2071 - val_accuracy: 0.9343
Epoch 58/100
462/462 [==============================] - 0s 151us/step - loss: 0.1024 - accuracy: 0.9567 - val_loss: 0.2337 - val_accuracy: 0.9192
Epoch 59/100
462/462 [==============================] - 0s 151us/step - loss: 0.1045 - accuracy: 0.9524 - val_loss: 0.1896 - val_accuracy: 0.9394
Epoch 60/100
462/462 [==============================] - 0s 155us/step - loss: 0.1112 - accuracy: 0.9502 - val_loss: 0.2500 - val_accuracy: 0.9343
Epoch 61/100
462/462 [==============================] - 0s 150us/step - loss: 0.1221 - accuracy: 0.9481 - val_loss: 0.2597 - val_accuracy: 0.8990
Epoch 62/100
462/462 [==============================] - 0s 149us/step - loss: 0.1386 - accuracy: 0.9329 - val_loss: 0.2468 - val_accuracy: 0.9242
Epoch 63/100
462/462 [==============================] - 0s 152us/step - loss: 0.1358 - accuracy: 0.9307 - val_loss: 0.2316 -

In [38]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 95.04%


In [139]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [140]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [141]:
y_prob2 = df_proba2[df_proba2['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[1.54025920e-02, 1.75332740e-02, 9.67064100e-01],
       [1.54025920e-02, 1.75332740e-02, 9.67064100e-01],
       [9.51918300e-01, 4.80610000e-02, 2.06755470e-05],
       [1.54025920e-02, 1.75332740e-02, 9.67064100e-01],
       [9.40524100e-01, 3.72458170e-02, 2.22301400e-02],
       [1.54025920e-02, 1.75332740e-02, 9.67064100e-01],
       [7.48730100e-01, 2.51247170e-01, 2.27176600e-05],
       [1.59230580e-03, 9.98407660e-01, 6.79445880e-09],
       [8.93430350e-01, 1.06569680e-01, 1.12411400e-09],
       [1.54025920e-02, 1.75332740e-02, 9.67064100e-01],
       [9.98249950e-01, 8.26172200e-04, 9.23807500e-04],
       [1.20846760e-04, 9.99879100e-01, 4.67794340e-09],
       [5.19184800e-01, 4.80815200e-01, 2.47268800e-10],
       [2.50108770e-04, 9.65306040e-01, 3.44439370e-02],
       [1.58350880e-02, 9.84163050e-01, 1.82845860e-06],
       [1.14487540e-04, 2.01551520e-03, 9.97869970e-01],
       [1.10075550e-03, 9.98898150e-01, 1.07561440e-06],
       [2.40822260e-06, 1.66191

In [142]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9841215182124273

In [143]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9841215182124273

In [144]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [145]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [146]:
dat3

,0,test
0,CFBREBSa110,0
1,CFBREBSa131,1
2,NRS148,2
3,NRS169,1
4,NRS073,0
...,...,...
193,NRS001,1
194,NRS191,0
195,NRS207,0
196,CA26,0


In [147]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 420us/step - loss: 8.6511 - accuracy: 0.3528 - val_loss: 3.9484 - val_accuracy: 0.4242
Epoch 2/100
462/462 [==============================] - 0s 256us/step - loss: 2.1768 - accuracy: 0.5346 - val_loss: 1.4929 - val_accuracy: 0.6313
Epoch 3/100
462/462 [==============================] - 0s 237us/step - loss: 1.1277 - accuracy: 0.6710 - val_loss: 1.4202 - val_accuracy: 0.6515
Epoch 4/100
462/462 [==============================] - 0s 202us/step - loss: 0.7432 - accuracy: 0.7294 - val_loss: 0.8195 - val_accuracy: 0.7121
Epoch 5/100
462/462 [==============================] - 0s 272us/step - loss: 0.5267 - accuracy: 0.7900 - val_loss: 0.7175 - val_accuracy: 0.7020
Epoch 6/100
462/462 [==============================] - 0s 244us/step - loss: 0.4634 - accuracy: 0.7922 - val_loss: 0.5882 - val_accuracy: 0.7879
Epoch 7/100
462/462 [==============================] - 0s 186us/step - loss: 0.4121 

Epoch 57/100
462/462 [==============================] - 0s 149us/step - loss: 0.1992 - accuracy: 0.9156 - val_loss: 0.2547 - val_accuracy: 0.9091
Epoch 58/100
462/462 [==============================] - 0s 128us/step - loss: 0.2106 - accuracy: 0.9069 - val_loss: 0.2941 - val_accuracy: 0.8687
Epoch 59/100
462/462 [==============================] - 0s 131us/step - loss: 0.2262 - accuracy: 0.9026 - val_loss: 0.3003 - val_accuracy: 0.8737
Epoch 60/100
462/462 [==============================] - 0s 265us/step - loss: 0.1846 - accuracy: 0.9264 - val_loss: 0.2575 - val_accuracy: 0.9040
Epoch 61/100
462/462 [==============================] - 0s 259us/step - loss: 0.1951 - accuracy: 0.9242 - val_loss: 0.2837 - val_accuracy: 0.8687
Epoch 62/100
462/462 [==============================] - 0s 154us/step - loss: 0.1975 - accuracy: 0.9026 - val_loss: 0.3095 - val_accuracy: 0.8485
Epoch 63/100
462/462 [==============================] - 0s 140us/step - loss: 0.1990 - accuracy: 0.9004 - val_loss: 0.2504 -

In [58]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

198/198 [==============================] - 0s 202us/step
over-sampling test accuracy: 91.92%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 1, 2, 1, 0, 1, 1, 2, 1, 0, 0, 2, 1, 0, 1, 2, 0, 2, 0, 2, 0, 2,
       1, 1, 0, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 1, 0, 0, 2, 2, 1, 0, 0,
       1, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 0,
       0, 1, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 0, 2, 2, 1, 2, 0, 0, 1, 0, 2,
       2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1,
       2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 0,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 0, 2, 1, 2, 1, 0, 2, 1, 1, 1, 2,
       1, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 0, 2, 0, 1, 2, 0, 1, 0, 1,
       1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 0])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,CFBREBSa110,0,0
1,CFBREBSa131,1,1
2,NRS148,2,2
3,NRS169,1,1
4,NRS073,0,0
...,...,...,...
193,NRS001,1,0
194,NRS191,0,1
195,NRS207,0,0
196,CA26,0,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.980640,0.019360,6.385857e-12
1,0.115201,0.884799,1.130102e-12
2,0.000027,0.007203,9.927697e-01
3,0.035446,0.774842,1.897115e-01
4,0.979925,0.020075,1.663623e-14
...,...,...,...
193,0.706597,0.293403,3.508671e-09
194,0.455959,0.544038,3.382804e-06
195,0.894158,0.105841,5.055511e-07
196,0.431733,0.568267,4.475438e-10


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p002ST.csv", index = False,
         header=None)

In [62]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 228us/step - loss: 0.1601 - accuracy: 0.9329 - val_loss: 0.2085 - val_accuracy: 0.9293
Epoch 2/100
462/462 [==============================] - 0s 195us/step - loss: 0.1619 - accuracy: 0.9416 - val_loss: 0.2004 - val_accuracy: 0.9242
Epoch 3/100
462/462 [==============================] - 0s 158us/step - loss: 0.1440 - accuracy: 0.9459 - val_loss: 0.2279 - val_accuracy: 0.9192
Epoch 4/100
462/462 [==============================] - 0s 132us/step - loss: 0.1451 - accuracy: 0.9481 - val_loss: 0.2119 - val_accuracy: 0.9343
Epoch 5/100
462/462 [==============================] - 0s 126us/step - loss: 0.1441 - accuracy: 0.9502 - val_loss: 0.2158 - val_accuracy: 0.9192
Epoch 6/100
462/462 [==============================] - 0s 146us/step - loss: 0.1693 - accuracy: 0.9307 - val_loss: 0.2632 - val_accuracy: 0.9141
Epoch 7/100
462/462 [==============================] - 0s 150us/step - loss: 0.1661 

Epoch 57/100
462/462 [==============================] - 0s 244us/step - loss: 0.1609 - accuracy: 0.9091 - val_loss: 0.1922 - val_accuracy: 0.9444
Epoch 58/100
462/462 [==============================] - 0s 202us/step - loss: 0.1376 - accuracy: 0.9481 - val_loss: 0.1995 - val_accuracy: 0.9394
Epoch 59/100
462/462 [==============================] - 0s 148us/step - loss: 0.1293 - accuracy: 0.9610 - val_loss: 0.1785 - val_accuracy: 0.9293
Epoch 60/100
462/462 [==============================] - 0s 156us/step - loss: 0.1312 - accuracy: 0.9351 - val_loss: 0.2218 - val_accuracy: 0.9091
Epoch 61/100
462/462 [==============================] - 0s 247us/step - loss: 0.1315 - accuracy: 0.9654 - val_loss: 0.2428 - val_accuracy: 0.8788
Epoch 62/100
462/462 [==============================] - 0s 207us/step - loss: 0.1374 - accuracy: 0.9351 - val_loss: 0.2252 - val_accuracy: 0.9343
Epoch 63/100
462/462 [==============================] - 0s 145us/step - loss: 0.1301 - accuracy: 0.9545 - val_loss: 0.1980 -

In [63]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 95.35%


In [148]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [149]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [150]:
y_prob3 = df_proba3[df_proba3['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.80639640e-01, 1.93604180e-02, 6.38585670e-12],
       [1.15201090e-01, 8.84798800e-01, 1.13010200e-12],
       [2.73399070e-05, 7.20297600e-03, 9.92769660e-01],
       [3.54463200e-02, 7.74842260e-01, 1.89711510e-01],
       [9.79925300e-01, 2.00747050e-02, 1.66362250e-14],
       [4.21098600e-06, 9.99995800e-01, 3.91039020e-12],
       [1.48725420e-03, 9.02751700e-01, 9.57610700e-02],
       [2.73399070e-05, 7.20297600e-03, 9.92769660e-01],
       [4.16497400e-03, 9.95834950e-01, 8.53640360e-08],
       [7.07982800e-01, 2.92017280e-01, 3.63820770e-09],
       [9.79925300e-01, 2.00747050e-02, 1.66362250e-14],
       [7.32985650e-03, 5.15077300e-02, 9.41162400e-01],
       [1.91200660e-01, 8.08799300e-01, 1.30500880e-08],
       [7.06596550e-01, 2.93403500e-01, 3.50867710e-09],
       [3.33864030e-01, 6.66120500e-01, 1.54581270e-05],
       [7.32985650e-03, 5.15077300e-02, 9.41162400e-01],
       [7.48909200e-01, 2.51090820e-01, 1.81729330e-08],
       [7.32985650e-03, 5.15077

In [151]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9796831955922864

In [152]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9796831955922864

In [153]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [154]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [155]:
dat4

,0,test
0,NRS110,1
1,NRS216,1
2,NRS386,1
3,CFBRSa25,0
4,BCH-SA-03,1
...,...,...
193,NRS216,1
194,CFBREBSa110,0
195,NRS148,2
196,GA27,0


In [156]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 403us/step - loss: 6.8908 - accuracy: 0.4199 - val_loss: 1.0015 - val_accuracy: 0.7525
Epoch 2/100
462/462 [==============================] - 0s 263us/step - loss: 1.0966 - accuracy: 0.7078 - val_loss: 0.9414 - val_accuracy: 0.7071
Epoch 3/100
462/462 [==============================] - 0s 261us/step - loss: 0.6503 - accuracy: 0.7662 - val_loss: 0.5916 - val_accuracy: 0.7677
Epoch 4/100
462/462 [==============================] - 0s 268us/step - loss: 0.5627 - accuracy: 0.7749 - val_loss: 0.6464 - val_accuracy: 0.7121
Epoch 5/100
462/462 [==============================] - 0s 241us/step - loss: 0.5552 - accuracy: 0.8117 - val_loss: 0.6187 - val_accuracy: 0.6970
Epoch 6/100
462/462 [==============================] - 0s 203us/step - loss: 0.4605 - accuracy: 0.8117 - val_loss: 0.6237 - val_accuracy: 0.6465
Epoch 7/100
462/462 [==============================] - 0s 191us/step - loss: 0.4456 

Epoch 57/100
462/462 [==============================] - 0s 284us/step - loss: 0.1866 - accuracy: 0.9307 - val_loss: 0.2470 - val_accuracy: 0.9242
Epoch 58/100
462/462 [==============================] - 0s 153us/step - loss: 0.1747 - accuracy: 0.9286 - val_loss: 0.2455 - val_accuracy: 0.9293
Epoch 59/100
462/462 [==============================] - 0s 151us/step - loss: 0.1694 - accuracy: 0.9264 - val_loss: 0.2664 - val_accuracy: 0.8535
Epoch 60/100
462/462 [==============================] - 0s 215us/step - loss: 0.1627 - accuracy: 0.9221 - val_loss: 0.2699 - val_accuracy: 0.8939
Epoch 61/100
462/462 [==============================] - 0s 222us/step - loss: 0.1792 - accuracy: 0.9156 - val_loss: 0.2402 - val_accuracy: 0.9343
Epoch 62/100
462/462 [==============================] - 0s 221us/step - loss: 0.1684 - accuracy: 0.9307 - val_loss: 0.2374 - val_accuracy: 0.9343
Epoch 63/100
462/462 [==============================] - 0s 302us/step - loss: 0.1583 - accuracy: 0.9372 - val_loss: 0.2480 -

In [71]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

198/198 [==============================] - 0s 158us/step
over-sampling test accuracy: 92.93%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 1, 0, 1, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 2, 1,
       1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 0, 0, 1, 1, 1,
       0, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 1, 1, 1, 2, 0, 2, 2,
       0, 0, 1, 2, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1, 0, 2, 1,
       1, 2, 2, 2, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 2, 0, 1,
       0, 2, 1, 2, 0, 0, 0, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 2,
       2, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 2, 1, 0, 1, 0, 2, 0, 0, 0, 1, 2, 0, 2, 2, 1, 0, 2, 0, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS110,1,1
1,NRS216,1,1
2,NRS386,1,1
3,CFBRSa25,0,0
4,BCH-SA-03,1,1
...,...,...,...
193,NRS216,1,1
194,CFBREBSa110,0,0
195,NRS148,2,2
196,GA27,0,0


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.064765,0.935234,2.409646e-07
1,0.269903,0.730082,1.490763e-05
2,0.450719,0.549281,7.583485e-08
3,0.690136,0.309864,8.500910e-08
4,0.325264,0.674735,1.414096e-08
...,...,...,...
193,0.269903,0.730082,1.490762e-05
194,0.989226,0.010774,3.609320e-10
195,0.000038,0.005363,9.945990e-01
196,0.997779,0.002221,2.268702e-13


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p002ST.csv", index = False,
         header=None)

In [75]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 248us/step - loss: 0.1653 - accuracy: 0.9307 - val_loss: 0.1715 - val_accuracy: 0.9343
Epoch 2/100
462/462 [==============================] - 0s 199us/step - loss: 0.1609 - accuracy: 0.9264 - val_loss: 0.2818 - val_accuracy: 0.8384
Epoch 3/100
462/462 [==============================] - 0s 183us/step - loss: 0.1330 - accuracy: 0.9481 - val_loss: 0.1859 - val_accuracy: 0.9343
Epoch 4/100
462/462 [==============================] - 0s 170us/step - loss: 0.1209 - accuracy: 0.9610 - val_loss: 0.1667 - val_accuracy: 0.9343
Epoch 5/100
462/462 [==============================] - 0s 157us/step - loss: 0.1172 - accuracy: 0.9545 - val_loss: 0.1817 - val_accuracy: 0.8939
Epoch 6/100
462/462 [==============================] - 0s 160us/step - loss: 0.1101 - accuracy: 0.9632 - val_loss: 0.1804 - val_accuracy: 0.9394
Epoch 7/100
462/462 [==============================] - 0s 221us/step - loss: 0.1392 

Epoch 57/100
462/462 [==============================] - 0s 330us/step - loss: 0.0949 - accuracy: 0.9654 - val_loss: 0.1445 - val_accuracy: 0.9495
Epoch 58/100
462/462 [==============================] - 0s 263us/step - loss: 0.1005 - accuracy: 0.9675 - val_loss: 0.1976 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 290us/step - loss: 0.1256 - accuracy: 0.9524 - val_loss: 0.1843 - val_accuracy: 0.9343
Epoch 60/100
462/462 [==============================] - 0s 274us/step - loss: 0.1104 - accuracy: 0.9481 - val_loss: 0.1713 - val_accuracy: 0.9293
Epoch 61/100
462/462 [==============================] - 0s 263us/step - loss: 0.1009 - accuracy: 0.9610 - val_loss: 0.1383 - val_accuracy: 0.9545
Epoch 62/100
462/462 [==============================] - 0s 243us/step - loss: 0.1003 - accuracy: 0.9589 - val_loss: 0.1455 - val_accuracy: 0.9444
Epoch 63/100
462/462 [==============================] - 0s 283us/step - loss: 0.1328 - accuracy: 0.9394 - val_loss: 0.1479 -

In [76]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 95.42%


In [157]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [158]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [159]:
y_prob4 = df_proba4[df_proba4['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[6.47654200e-02, 9.35234370e-01, 2.40964620e-07],
       [2.69903120e-01, 7.30082040e-01, 1.49076310e-05],
       [4.50718880e-01, 5.49281000e-01, 7.58348550e-08],
       [6.90135600e-01, 3.09864300e-01, 8.50091000e-08],
       [3.25264480e-01, 6.74735400e-01, 1.41409610e-08],
       [6.47654200e-02, 9.35234370e-01, 2.40964620e-07],
       [2.37929900e-05, 5.47145140e-05, 9.99921440e-01],
       [2.37929900e-05, 5.47145140e-05, 9.99921440e-01],
       [2.37929900e-05, 5.47145140e-05, 9.99921440e-01],
       [8.10910640e-01, 1.87278970e-01, 1.81037770e-03],
       [3.83570880e-05, 5.36263500e-03, 9.94599000e-01],
       [9.17054060e-01, 8.29457700e-02, 2.11111550e-07],
       [9.04337170e-01, 9.55729400e-02, 8.99474760e-05],
       [6.93428800e-01, 3.06570980e-01, 2.28927950e-07],
       [6.90135600e-01, 3.09864300e-01, 8.50091000e-08],
       [2.37929900e-05, 5.47145140e-05, 9.99921440e-01],
       [9.86825640e-01, 1.30792510e-02, 9.52231060e-05],
       [2.37929900e-05, 5.47145

In [160]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9791475359657178

In [161]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9791475359657178

In [163]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9840832568105295

In [164]:
np.std(ovos)

0.00570454278101484

In [165]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9840832568105295

In [166]:
np.std(ovrs)

0.00570454278101484

In [77]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [78]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 92.80%


In [79]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.00550367440540207


In [80]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [81]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 95.14%


In [82]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0026521664


In [167]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [168]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [169]:
dat5

,0,test
0,NRS241,1
1,NRS148,2
2,NRS255,1
3,NRS214,0
4,NRS148,2
...,...,...
193,CFBRSa30,0
194,NRS266,1
195,SR4152,0
196,NRS109,2


In [170]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [70]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [71]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 997us/step - loss: 5.0409 - accuracy: 0.5476 - val_loss: 2.1682 - val_accuracy: 0.6465
Epoch 2/100
462/462 [==============================] - 0s 344us/step - loss: 3.8850 - accuracy: 0.5866 - val_loss: 1.5057 - val_accuracy: 0.6616
Epoch 3/100
462/462 [==============================] - 0s 385us/step - loss: 3.5170 - accuracy: 0.6104 - val_loss: 1.3378 - val_accuracy: 0.6667
Epoch 4/100
462/462 [==============================] - 0s 277us/step - loss: 3.2984 - accuracy: 0.6126 - val_loss: 1.2692 - val_accuracy: 0.6717
Epoch 5/100
462/462 [==============================] - 0s 405us/step - loss: 3.2609 - accuracy: 0.6082 - val_loss: 1.2305 - val_accuracy: 0.6717
Epoch 6/100
462/462 [==============================] - 0s 340us/step - loss: 3.1466 - accuracy: 0.6255 - val_loss: 1.2937 - val_accuracy: 0.7020
Epoch 7/100
462/462 [==============================] - 0s 350us/step - loss: 3.1494 

Epoch 57/100
462/462 [==============================] - 0s 287us/step - loss: 1.2077 - accuracy: 0.7706 - val_loss: 0.6372 - val_accuracy: 0.8636
Epoch 58/100
462/462 [==============================] - 0s 225us/step - loss: 1.4994 - accuracy: 0.7576 - val_loss: 0.8394 - val_accuracy: 0.8788
Epoch 59/100
462/462 [==============================] - 0s 243us/step - loss: 1.4125 - accuracy: 0.7424 - val_loss: 0.6487 - val_accuracy: 0.8737
Epoch 60/100
462/462 [==============================] - 0s 242us/step - loss: 1.5338 - accuracy: 0.7165 - val_loss: 0.6797 - val_accuracy: 0.8939
Epoch 61/100
462/462 [==============================] - 0s 234us/step - loss: 1.6860 - accuracy: 0.7511 - val_loss: 0.7380 - val_accuracy: 0.8535
Epoch 62/100
462/462 [==============================] - 0s 235us/step - loss: 1.4737 - accuracy: 0.7489 - val_loss: 0.7607 - val_accuracy: 0.8687
Epoch 63/100
462/462 [==============================] - 0s 251us/step - loss: 1.2698 - accuracy: 0.7381 - val_loss: 0.9811 -

In [218]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

198/198 [==============================] - 0s 160us/step
over-sampling test accuracy: 86.36%


In [73]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([1, 2, 1, 0, 2, 1, 2, 2, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0, 1, 2,
       0, 0, 0, 2, 2, 0, 1, 1, 1, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 2, 0, 2,
       2, 2, 0, 1, 2, 1, 1, 0, 0, 2, 0, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 0,
       2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 2, 1, 2, 0, 1, 2, 0, 1, 1, 1, 2, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 2, 2, 2, 1, 2, 2, 0, 1, 1, 2, 1,
       1, 1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 2, 1, 1, 1, 1, 0, 2,
       0, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 0, 2, 0, 1, 0, 2, 1, 2, 0, 1, 2,
       2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 2, 1])

In [74]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS241,1,1
1,NRS148,2,2
2,NRS255,1,1
3,NRS214,0,0
4,NRS148,2,2
...,...,...,...
193,CFBRSa30,0,0
194,NRS266,1,1
195,SR4152,0,0
196,NRS109,2,2


In [75]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [76]:
dat_proba5

,0,1,2
0,1.342914e-03,9.986569e-01,2.348628e-07
1,5.170289e-08,1.017893e-07,9.999999e-01
2,1.780311e-07,9.999999e-01,2.544841e-12
3,1.000000e+00,2.203547e-10,5.688883e-15
4,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...
193,1.000000e+00,5.251366e-13,1.614020e-18
194,1.216973e-08,1.000000e+00,2.654322e-14
195,9.943394e-01,5.660585e-03,4.075664e-13
196,7.450425e-21,9.585720e-12,1.000000e+00


In [77]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [78]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p002ST.csv", index = False,
         header=None)

In [222]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 382us/step - loss: 1.4464 - accuracy: 0.7186 - val_loss: 1.4367 - val_accuracy: 0.7778
Epoch 2/100
462/462 [==============================] - 0s 288us/step - loss: 1.3780 - accuracy: 0.7035 - val_loss: 1.4334 - val_accuracy: 0.8485
Epoch 3/100
462/462 [==============================] - 0s 299us/step - loss: 1.2255 - accuracy: 0.7446 - val_loss: 0.9547 - val_accuracy: 0.8788
Epoch 4/100
462/462 [==============================] - 0s 268us/step - loss: 1.3377 - accuracy: 0.7316 - val_loss: 1.1604 - val_accuracy: 0.8384
Epoch 5/100
462/462 [==============================] - 0s 256us/step - loss: 1.1934 - accuracy: 0.7316 - val_loss: 1.1594 - val_accuracy: 0.8687
Epoch 6/100
462/462 [==============================] - 0s 359us/step - loss: 1.2729 - accuracy: 0.7446 - val_loss: 1.3179 - val_accuracy: 0.8535
Epoch 7/100
462/462 [==============================] - 0s 252us/step - loss: 1.1330 

Epoch 57/100
462/462 [==============================] - 0s 410us/step - loss: 1.2665 - accuracy: 0.7294 - val_loss: 1.2587 - val_accuracy: 0.8384
Epoch 58/100
462/462 [==============================] - 0s 389us/step - loss: 1.2981 - accuracy: 0.7424 - val_loss: 0.8914 - val_accuracy: 0.8384
Epoch 59/100
462/462 [==============================] - 0s 374us/step - loss: 1.1904 - accuracy: 0.7165 - val_loss: 1.2400 - val_accuracy: 0.8333
Epoch 60/100
462/462 [==============================] - 0s 341us/step - loss: 1.1297 - accuracy: 0.7468 - val_loss: 1.5495 - val_accuracy: 0.8131
Epoch 61/100
462/462 [==============================] - 0s 315us/step - loss: 1.2268 - accuracy: 0.7468 - val_loss: 1.8264 - val_accuracy: 0.8232
Epoch 62/100
462/462 [==============================] - 0s 321us/step - loss: 1.4871 - accuracy: 0.7056 - val_loss: 2.3550 - val_accuracy: 0.8030
Epoch 63/100
462/462 [==============================] - 0s 393us/step - loss: 1.2379 - accuracy: 0.7424 - val_loss: 1.1821 -

In [223]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 73.07%


In [171]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [172]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [173]:
y_prob5 = df_proba5[df_proba5['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.34291400e-03, 9.98656870e-01, 2.34862780e-07],
       [5.17028940e-08, 1.01789280e-07, 9.99999900e-01],
       [1.78031120e-07, 9.99999900e-01, 2.54484100e-12],
       [1.00000000e+00, 2.20354690e-10, 5.68888330e-15],
       [5.17028940e-08, 1.01789280e-07, 9.99999900e-01],
       [3.32041760e-03, 9.96389030e-01, 2.90551020e-04],
       [7.45042500e-21, 9.58572000e-12, 1.00000000e+00],
       [9.92191200e-09, 1.72646430e-08, 1.00000000e+00],
       [3.32041760e-03, 9.96389030e-01, 2.90551020e-04],
       [8.46151350e-01, 1.53848700e-01, 1.73614970e-08],
       [5.17028940e-08, 1.01789280e-07, 9.99999900e-01],
       [9.92191200e-09, 1.72646430e-08, 1.00000000e+00],
       [7.45042500e-21, 9.58572000e-12, 1.00000000e+00],
       [3.27969710e-03, 9.96719400e-01, 8.87750840e-07],
       [1.00000000e+00, 5.25136630e-13, 1.61401960e-18],
       [7.45042500e-21, 9.58572000e-12, 1.00000000e+00],
       [5.17028940e-08, 1.01789280e-07, 9.99999900e-01],
       [3.96341230e-09, 1.67940

In [174]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9745179063360881

In [175]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9745179063360881

In [176]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [177]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [178]:
dat6

,0,test
0,NRS209,2
1,NRS386,1
2,NRS148,2
3,NRS178,0
4,NRS237,0
...,...,...
193,NRS209,2
194,NRS002,0
195,NRS109,2
196,BCH-SA-03,1


In [179]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [83]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 772us/step - loss: 6.1974 - accuracy: 0.3506 - val_loss: 1.3443 - val_accuracy: 0.4091
Epoch 2/100
462/462 [==============================] - 0s 367us/step - loss: 3.7813 - accuracy: 0.5844 - val_loss: 0.9154 - val_accuracy: 0.6970
Epoch 3/100
462/462 [==============================] - 0s 319us/step - loss: 3.7779 - accuracy: 0.6169 - val_loss: 0.8784 - val_accuracy: 0.7374
Epoch 4/100
462/462 [==============================] - 0s 262us/step - loss: 3.6844 - accuracy: 0.6364 - val_loss: 0.8273 - val_accuracy: 0.7424
Epoch 5/100
462/462 [==============================] - 0s 314us/step - loss: 2.9144 - accuracy: 0.6797 - val_loss: 0.8410 - val_accuracy: 0.7626
Epoch 6/100
462/462 [==============================] - 0s 311us/step - loss: 3.0821 - accuracy: 0.6537 - val_loss: 0.7675 - val_accuracy: 0.7374
Epoch 7/100
462/462 [==============================] - 0s 261us/step - loss: 3.4016 

Epoch 57/100
462/462 [==============================] - 0s 378us/step - loss: 1.4179 - accuracy: 0.7078 - val_loss: 0.8373 - val_accuracy: 0.8485
Epoch 58/100
462/462 [==============================] - 0s 394us/step - loss: 1.8928 - accuracy: 0.6667 - val_loss: 1.1149 - val_accuracy: 0.8283
Epoch 59/100
462/462 [==============================] - 0s 387us/step - loss: 1.6061 - accuracy: 0.7338 - val_loss: 0.9136 - val_accuracy: 0.8384
Epoch 60/100
462/462 [==============================] - 0s 272us/step - loss: 1.6161 - accuracy: 0.7316 - val_loss: 1.5014 - val_accuracy: 0.8232
Epoch 61/100
462/462 [==============================] - 0s 248us/step - loss: 1.4598 - accuracy: 0.7381 - val_loss: 1.4240 - val_accuracy: 0.8434
Epoch 62/100
462/462 [==============================] - 0s 259us/step - loss: 1.6996 - accuracy: 0.7316 - val_loss: 0.8404 - val_accuracy: 0.9091
Epoch 63/100
462/462 [==============================] - 0s 291us/step - loss: 1.1171 - accuracy: 0.7468 - val_loss: 1.0528 -

In [231]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

198/198 [==============================] - 0s 157us/step
over-sampling test accuracy: 88.89%


In [86]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0,
       0, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2,
       2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 2, 0, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2,
       0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 0, 2, 0, 1, 0, 0,
       1, 2, 2, 0, 1, 0, 2, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 1, 0, 1,
       2, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 0, 2, 0, 1, 0, 1, 0,
       1, 1, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 2,
       1, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1,
       0, 1, 1, 0, 0, 2, 1, 0, 1, 1, 1, 0, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1])

In [87]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS209,2,2
1,NRS386,1,1
2,NRS148,2,2
3,NRS178,0,1
4,NRS237,0,1
...,...,...,...
193,NRS209,2,2
194,NRS002,0,1
195,NRS109,2,2
196,BCH-SA-03,1,1


In [88]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [89]:
dat_proba6

,0,1,2
0,1.790400e-08,4.141849e-08,1.000000e+00
1,5.739934e-04,9.994259e-01,6.773014e-08
2,5.286934e-09,1.269109e-08,1.000000e+00
3,6.494936e-12,1.000000e+00,2.537080e-25
4,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...
193,1.790400e-08,4.141849e-08,1.000000e+00
194,3.156084e-02,9.684391e-01,5.647518e-09
195,1.503430e-08,3.722075e-08,1.000000e+00
196,7.681937e-03,9.922962e-01,2.191572e-05


In [90]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [91]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p002ST.csv", index = False,
         header=None)

In [235]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 415us/step - loss: 0.9860 - accuracy: 0.7619 - val_loss: 0.7078 - val_accuracy: 0.8737
Epoch 2/100
462/462 [==============================] - 0s 391us/step - loss: 0.8170 - accuracy: 0.7965 - val_loss: 0.7615 - val_accuracy: 0.8889
Epoch 3/100
462/462 [==============================] - 0s 398us/step - loss: 1.0788 - accuracy: 0.7316 - val_loss: 0.7874 - val_accuracy: 0.8889
Epoch 4/100
462/462 [==============================] - 0s 329us/step - loss: 1.3773 - accuracy: 0.7208 - val_loss: 0.7465 - val_accuracy: 0.8889
Epoch 5/100
462/462 [==============================] - 0s 315us/step - loss: 1.1219 - accuracy: 0.7554 - val_loss: 0.6550 - val_accuracy: 0.8889
Epoch 6/100
462/462 [==============================] - 0s 287us/step - loss: 1.0661 - accuracy: 0.7532 - val_loss: 0.7018 - val_accuracy: 0.8838
Epoch 7/100
462/462 [==============================] - 0s 291us/step - loss: 1.1267 

Epoch 57/100
462/462 [==============================] - 0s 462us/step - loss: 1.1008 - accuracy: 0.7208 - val_loss: 1.6672 - val_accuracy: 0.8889
Epoch 58/100
462/462 [==============================] - 0s 329us/step - loss: 1.2066 - accuracy: 0.7359 - val_loss: 0.8819 - val_accuracy: 0.8788
Epoch 59/100
462/462 [==============================] - 0s 260us/step - loss: 1.1541 - accuracy: 0.7338 - val_loss: 1.1870 - val_accuracy: 0.8636
Epoch 60/100
462/462 [==============================] - 0s 260us/step - loss: 1.4143 - accuracy: 0.7532 - val_loss: 1.1872 - val_accuracy: 0.8687
Epoch 61/100
462/462 [==============================] - 0s 505us/step - loss: 1.2395 - accuracy: 0.7511 - val_loss: 1.0338 - val_accuracy: 0.8636
Epoch 62/100
462/462 [==============================] - 0s 695us/step - loss: 1.1043 - accuracy: 0.7316 - val_loss: 0.9848 - val_accuracy: 0.8990
Epoch 63/100
462/462 [==============================] - 0s 320us/step - loss: 1.3341 - accuracy: 0.7468 - val_loss: 2.1759 -

In [236]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 75.08%


In [180]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [181]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [182]:
y_prob6 = df_proba6[df_proba6['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.79040020e-08, 4.14184900e-08, 1.00000000e+00],
       [5.73993370e-04, 9.99425900e-01, 6.77301400e-08],
       [5.28693440e-09, 1.26910935e-08, 1.00000000e+00],
       [6.49493600e-12, 1.00000000e+00, 2.53707970e-25],
       [5.70109800e-02, 9.39920370e-01, 3.06858290e-03],
       [1.54960120e-08, 1.00000000e+00, 7.12019400e-18],
       [7.68193000e-03, 9.92296160e-01, 2.19156830e-05],
       [1.79040020e-08, 4.14184900e-08, 1.00000000e+00],
       [1.54960120e-08, 1.00000000e+00, 7.12019400e-18],
       [5.70109800e-02, 9.39920370e-01, 3.06858290e-03],
       [5.73993370e-04, 9.99425900e-01, 6.77301400e-08],
       [2.52260520e-06, 9.99997500e-01, 5.17447700e-13],
       [9.84662600e-06, 9.99990100e-01, 3.82570170e-10],
       [2.27302350e-09, 1.00000000e+00, 1.90519700e-21],
       [8.51542300e-01, 1.48454870e-01, 2.79731240e-06],
       [7.68193000e-03, 9.92296160e-01, 2.19156830e-05],
       [9.99984600e-01, 1.51759790e-05, 2.93080030e-07],
       [1.84558290e-04, 9.99815

In [183]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.966215182124273

In [184]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.966215182124273

In [185]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [186]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [187]:
dat7

,0,test
0,NRS209,2
1,BCH-SA-09,1
2,NRS224,0
3,NRS209,2
4,NRS235,1
...,...,...
193,NRS209,2
194,CFBREBSa131,1
195,CFBREBSa103,0
196,NRS188,1


In [188]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [96]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 808us/step - loss: 6.3052 - accuracy: 0.4177 - val_loss: 2.6459 - val_accuracy: 0.6313
Epoch 2/100
462/462 [==============================] - 0s 478us/step - loss: 4.2037 - accuracy: 0.5455 - val_loss: 1.6030 - val_accuracy: 0.7222
Epoch 3/100
462/462 [==============================] - 0s 469us/step - loss: 3.6331 - accuracy: 0.5649 - val_loss: 1.2393 - val_accuracy: 0.7071
Epoch 4/100
462/462 [==============================] - 0s 473us/step - loss: 3.4023 - accuracy: 0.5887 - val_loss: 1.2671 - val_accuracy: 0.6616
Epoch 5/100
462/462 [==============================] - 0s 275us/step - loss: 3.3145 - accuracy: 0.6234 - val_loss: 1.1075 - val_accuracy: 0.6919
Epoch 6/100
462/462 [==============================] - 0s 439us/step - loss: 3.6862 - accuracy: 0.5844 - val_loss: 1.3027 - val_accuracy: 0.6667
Epoch 7/100
462/462 [==============================] - 0s 832us/step - loss: 3.0736 

Epoch 57/100
462/462 [==============================] - 0s 253us/step - loss: 1.8372 - accuracy: 0.6905 - val_loss: 1.0593 - val_accuracy: 0.7828
Epoch 58/100
462/462 [==============================] - 0s 249us/step - loss: 1.9578 - accuracy: 0.6926 - val_loss: 0.9821 - val_accuracy: 0.8283
Epoch 59/100
462/462 [==============================] - 0s 303us/step - loss: 1.5771 - accuracy: 0.7294 - val_loss: 0.8123 - val_accuracy: 0.8131
Epoch 60/100
462/462 [==============================] - 0s 359us/step - loss: 1.4708 - accuracy: 0.7143 - val_loss: 0.6732 - val_accuracy: 0.8636
Epoch 61/100
462/462 [==============================] - 0s 354us/step - loss: 1.8062 - accuracy: 0.6883 - val_loss: 0.7497 - val_accuracy: 0.8889
Epoch 62/100
462/462 [==============================] - 0s 389us/step - loss: 1.4919 - accuracy: 0.7100 - val_loss: 0.6637 - val_accuracy: 0.8990
Epoch 63/100
462/462 [==============================] - 0s 353us/step - loss: 1.6543 - accuracy: 0.7403 - val_loss: 0.7113 -

In [244]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))


198/198 [==============================] - 0s 143us/step
over-sampling test accuracy: 84.85%


In [99]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([2, 1, 0, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0,
       2, 0, 0, 2, 1, 0, 1, 0, 1, 2, 0, 2, 2, 2, 1, 2, 0, 0, 2, 1, 2, 0,
       2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 1, 0,
       0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1,
       1, 0, 2, 2, 2, 2, 2, 1, 0, 2, 1, 0, 1, 2, 0, 0, 2, 1, 2, 0, 1, 1,
       2, 2, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0, 1, 2,
       0, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 2, 1, 2, 0, 2, 2, 0, 2,
       0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2])

In [100]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS209,2,2
1,BCH-SA-09,1,1
2,NRS224,0,0
3,NRS209,2,2
4,NRS235,1,1
...,...,...,...
193,NRS209,2,2
194,CFBREBSa131,1,0
195,CFBREBSa103,0,0
196,NRS188,1,1


In [101]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [102]:
dat_proba7

,0,1,2
0,8.300497e-12,1.036520e-09,1.000000e+00
1,1.137139e-06,9.999988e-01,2.067601e-09
2,1.000000e+00,2.093109e-31,0.000000e+00
3,8.300497e-12,1.036520e-09,1.000000e+00
4,2.243512e-02,9.774035e-01,1.615106e-04
...,...,...,...
193,8.300497e-12,1.036520e-09,1.000000e+00
194,1.000000e+00,1.609111e-29,0.000000e+00
195,1.000000e+00,3.554443e-18,8.619599e-28
196,1.033436e-24,1.000000e+00,4.421581e-35


In [103]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [104]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p002ST.csv", index = False,
         header=None)

In [248]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 378us/step - loss: 1.6578 - accuracy: 0.6905 - val_loss: 1.0423 - val_accuracy: 0.8535
Epoch 2/100
462/462 [==============================] - 0s 327us/step - loss: 1.5900 - accuracy: 0.7186 - val_loss: 0.8690 - val_accuracy: 0.8485
Epoch 3/100
462/462 [==============================] - 0s 278us/step - loss: 1.5007 - accuracy: 0.7229 - val_loss: 1.0957 - val_accuracy: 0.8485
Epoch 4/100
462/462 [==============================] - 0s 266us/step - loss: 1.5462 - accuracy: 0.7208 - val_loss: 1.0411 - val_accuracy: 0.8636
Epoch 5/100
462/462 [==============================] - 0s 326us/step - loss: 1.6530 - accuracy: 0.6905 - val_loss: 0.9831 - val_accuracy: 0.8485
Epoch 6/100
462/462 [==============================] - 0s 338us/step - loss: 1.5349 - accuracy: 0.7446 - val_loss: 1.0328 - val_accuracy: 0.8384
Epoch 7/100
462/462 [==============================] - 0s 467us/step - loss: 1.4137 

Epoch 57/100
462/462 [==============================] - 0s 414us/step - loss: 1.5539 - accuracy: 0.7316 - val_loss: 1.6348 - val_accuracy: 0.8636
Epoch 58/100
462/462 [==============================] - 0s 406us/step - loss: 1.7640 - accuracy: 0.7143 - val_loss: 1.4207 - val_accuracy: 0.8434
Epoch 59/100
462/462 [==============================] - 0s 462us/step - loss: 1.5907 - accuracy: 0.7403 - val_loss: 1.5577 - val_accuracy: 0.8434
Epoch 60/100
462/462 [==============================] - 0s 327us/step - loss: 1.5552 - accuracy: 0.7359 - val_loss: 1.2601 - val_accuracy: 0.8586
Epoch 61/100
462/462 [==============================] - 0s 283us/step - loss: 1.6382 - accuracy: 0.7078 - val_loss: 1.4485 - val_accuracy: 0.8889
Epoch 62/100
462/462 [==============================] - 0s 338us/step - loss: 1.7584 - accuracy: 0.7381 - val_loss: 1.4144 - val_accuracy: 0.8434
Epoch 63/100
462/462 [==============================] - 0s 343us/step - loss: 1.5909 - accuracy: 0.7424 - val_loss: 1.5324 -

In [269]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 72.19%


In [189]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [190]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [191]:
y_prob7 = df_proba7[df_proba7['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[8.30049740e-12, 1.03651990e-09, 1.00000000e+00],
       [1.13713930e-06, 9.99998800e-01, 2.06760080e-09],
       [1.00000000e+00, 2.09310950e-31, 0.00000000e+00],
       [8.30049740e-12, 1.03651990e-09, 1.00000000e+00],
       [2.24351250e-02, 9.77403460e-01, 1.61510590e-04],
       [9.95281500e-01, 4.71842240e-03, 4.87819420e-14],
       [4.88350900e-01, 5.11649000e-01, 6.25775800e-08],
       [8.30049740e-12, 1.03651990e-09, 1.00000000e+00],
       [7.65375540e-11, 1.20261580e-09, 1.00000000e+00],
       [2.24351250e-02, 9.77403460e-01, 1.61510590e-04],
       [4.09843400e-06, 9.99995950e-01, 4.15722460e-20],
       [1.00000000e+00, 5.68623340e-31, 0.00000000e+00],
       [1.00000000e+00, 8.43185700e-34, 0.00000000e+00],
       [1.00000000e+00, 4.08136640e-25, 7.79217000e-38],
       [4.88350900e-01, 5.11649000e-01, 6.25775800e-08],
       [8.30124700e-22, 9.95855700e-21, 1.00000000e+00],
       [7.65375540e-11, 1.20261580e-09, 1.00000000e+00],
       [8.30049740e-12, 1.03651

In [192]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9410391796755433

In [193]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9410391796755433

In [194]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [195]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [196]:
dat8

,0,test
0,CFBREBSa116,0
1,NRS214,0
2,NRS148,2
3,NRS148,2
4,NRS148,2
...,...,...
193,NRS148,2
194,NRS054,0
195,NRS109,2
196,NRS216,1


In [197]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [109]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 781us/step - loss: 4.9098 - accuracy: 0.5498 - val_loss: 1.7761 - val_accuracy: 0.7727
Epoch 2/100
462/462 [==============================] - 0s 404us/step - loss: 4.1676 - accuracy: 0.5931 - val_loss: 1.7427 - val_accuracy: 0.6869
Epoch 3/100
462/462 [==============================] - 0s 423us/step - loss: 3.8215 - accuracy: 0.6169 - val_loss: 1.0949 - val_accuracy: 0.7424
Epoch 4/100
462/462 [==============================] - 0s 364us/step - loss: 3.4556 - accuracy: 0.6429 - val_loss: 1.1181 - val_accuracy: 0.7323
Epoch 5/100
462/462 [==============================] - 0s 439us/step - loss: 3.1721 - accuracy: 0.6450 - val_loss: 1.1670 - val_accuracy: 0.6768
Epoch 6/100
462/462 [==============================] - 0s 257us/step - loss: 3.1381 - accuracy: 0.6558 - val_loss: 1.2747 - val_accuracy: 0.7828
Epoch 7/100
462/462 [==============================] - 0s 367us/step - loss: 3.0135 

In [257]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))


198/198 [==============================] - 0s 165us/step
over-sampling test accuracy: 91.92%


In [112]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([0, 0, 2, 2, 2, 1, 2, 0, 0, 1, 0, 2, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 0, 2, 0, 0, 0, 1, 2, 0, 2, 2,
       0, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 2, 0, 2, 0, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 0, 1, 0,
       2, 1, 0, 2, 2, 2, 0, 1, 0, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 0,
       0, 2, 2, 2, 0, 0, 2, 2, 1, 0, 0, 1, 2, 0, 0, 2, 1, 0, 0, 1, 2, 2,
       2, 2, 2, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 1, 2, 2,
       2, 0, 2, 1, 0, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 2, 0, 2, 0, 2, 1, 0])

In [113]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,CFBREBSa116,0,0
1,NRS214,0,0
2,NRS148,2,2
3,NRS148,2,2
4,NRS148,2,2
...,...,...,...
193,NRS148,2,2
194,NRS054,0,0
195,NRS109,2,2
196,NRS216,1,1


In [114]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [115]:
dat_proba8

,0,1,2
0,9.676203e-01,3.237956e-02,1.480167e-07
1,1.000000e+00,6.534852e-11,2.250731e-18
2,3.948571e-11,2.839096e-07,9.999998e-01
3,3.948571e-11,2.839096e-07,9.999998e-01
4,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...
193,3.948571e-11,2.839091e-07,9.999998e-01
194,1.000000e+00,4.539984e-09,2.018661e-15
195,1.170756e-15,7.536062e-10,1.000000e+00
196,6.065643e-03,9.939343e-01,7.595704e-18


In [116]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [117]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p002ST.csv", index = False,
         header=None)

In [261]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 366us/step - loss: 1.2137 - accuracy: 0.7489 - val_loss: 1.0553 - val_accuracy: 0.8333
Epoch 2/100
462/462 [==============================] - 0s 397us/step - loss: 1.2706 - accuracy: 0.7381 - val_loss: 1.1689 - val_accuracy: 0.8586
Epoch 3/100
462/462 [==============================] - 0s 379us/step - loss: 1.1069 - accuracy: 0.7359 - val_loss: 0.8190 - val_accuracy: 0.8636
Epoch 4/100
462/462 [==============================] - 0s 438us/step - loss: 1.3298 - accuracy: 0.7446 - val_loss: 1.1741 - val_accuracy: 0.8838
Epoch 5/100
462/462 [==============================] - 0s 366us/step - loss: 1.2086 - accuracy: 0.7511 - val_loss: 1.0793 - val_accuracy: 0.8838
Epoch 6/100
462/462 [==============================] - 0s 318us/step - loss: 1.2321 - accuracy: 0.7641 - val_loss: 1.2733 - val_accuracy: 0.8485
Epoch 7/100
462/462 [==============================] - 0s 311us/step - loss: 1.3012 

Epoch 57/100
462/462 [==============================] - 0s 534us/step - loss: 1.2030 - accuracy: 0.7554 - val_loss: 0.7067 - val_accuracy: 0.8636
Epoch 58/100
462/462 [==============================] - 0s 378us/step - loss: 1.3228 - accuracy: 0.7424 - val_loss: 0.6180 - val_accuracy: 0.8182
Epoch 59/100
462/462 [==============================] - 0s 517us/step - loss: 1.3046 - accuracy: 0.7597 - val_loss: 0.5951 - val_accuracy: 0.8939
Epoch 60/100
462/462 [==============================] - 0s 351us/step - loss: 1.3086 - accuracy: 0.7013 - val_loss: 0.5394 - val_accuracy: 0.8485
Epoch 61/100
462/462 [==============================] - 0s 364us/step - loss: 1.1036 - accuracy: 0.7468 - val_loss: 0.5890 - val_accuracy: 0.8485
Epoch 62/100
462/462 [==============================] - 0s 386us/step - loss: 1.1340 - accuracy: 0.7273 - val_loss: 0.5131 - val_accuracy: 0.8333
Epoch 63/100
462/462 [==============================] - 0s 383us/step - loss: 1.0736 - accuracy: 0.7446 - val_loss: 0.5768 -

In [262]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 73.99%


In [198]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [199]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [200]:
y_prob8 = df_proba8[df_proba8['phage']=='p002ykpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[9.6762030e-01, 3.2379560e-02, 1.4801665e-07],
       [1.0000000e+00, 6.5348520e-11, 2.2507305e-18],
       [3.9485713e-11, 2.8390960e-07, 9.9999976e-01],
       [3.9485713e-11, 2.8390960e-07, 9.9999976e-01],
       [3.9485713e-11, 2.8390960e-07, 9.9999976e-01],
       [6.0656667e-03, 9.9393433e-01, 7.5956460e-18],
       [3.9485713e-11, 2.8390960e-07, 9.9999976e-01],
       [9.9916040e-01, 8.3955505e-04, 6.0683490e-11],
       [9.9999990e-01, 7.1966340e-08, 1.6942623e-13],
       [1.8313165e-03, 9.9816200e-01, 6.7235706e-06],
       [9.9802930e-01, 1.9706318e-03, 8.2175170e-10],
       [1.1707652e-15, 7.5361050e-10, 1.0000000e+00],
       [1.8313165e-03, 9.9816200e-01, 6.7235706e-06],
       [1.0000000e+00, 2.2062680e-10, 1.5829654e-17],
       [9.4680405e-01, 5.3195810e-02, 7.0303910e-08],
       [8.7235700e-12, 9.9335040e-08, 9.9999990e-01],
       [2.9259754e-06, 8.7246345e-04, 9.9912460e-01],
       [3.9485713e-11, 2.8390960e-07, 9.9999976e-01],
       [9.9013734e-01, 9.841

In [201]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9608394551576369

In [202]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9608394551576369

In [203]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9606529308233853

In [204]:
np.std(ovos2)

0.012327917615303271

In [205]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9606529308233853

In [206]:
np.std(ovrs2)

0.012327917615303271

In [263]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [264]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 88.01%


In [265]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.02681407495931643


In [266]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [267]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 73.58%


In [268]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.010700323
